## Предобработка данных и логистическая регрессия для задачи бинарной классификации (продолжение)

Сегодня мы продолжим знакомиться с основными техниками предобработки данных.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import pickle
from sklearn.linear_model import LogisticRegression as LR
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectFromModel
from sklearn.manifold import TSNE
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

%matplotlib inline

## Загрузка данных.

Для начала загрузим и инициализируем все необходимые [данные](https://yadi.sk/d/lvNm3uNm3LGvG3).

In [ ]:
with open('saved_data.pickle', 'rb') as f:
    X_train_real_scaled = pickle.load(f)
    X_test_real_scaled = pickle.load(f)
    X_train_cat_oh = pickle.load(f)
    X_test_cat_oh = pickle.load(f)
    y_train = pickle.load(f)
    y_test = pickle.load(f)
    
param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
cv=3

def plot_scores(optimizer):
    scores = [[item[0]['C'], 
               item[1], 
               (np.sum((item[2]-item[1])**2)/(item[2].size-1))**0.5] for item in optimizer.grid_scores_]
    scores = np.array(scores)
    plt.semilogx(scores[:,0], scores[:,1])
    plt.fill_between(scores[:,0], scores[:,1]-scores[:,2], 
                                  scores[:,1]+scores[:,2], alpha=0.3)
    plt.show()

**Задание**. Выведите общее количество анализируемых объектов в обучающей выборке и количество признаков в датасете.

In [ ]:
# место для вашего кода

# Визуализация данных в пространстве признаков

Будем рассматривать наши данные как точки в N-мерном пространстве. В таком случае, есть необходимость посмотреть на это множество точек. Тогда, возможно, мы сможем увидеть где нужно провести ту самую разделяющую гиперплоскость. Проблема в том, что пространство слишком большой размерности. Разумным решением является проецирование всего пространства признаков на двумерное (трехмерное) подпространство. Одним из самых популярных метод такого проецирования является t-SNE (t-distributed Stochastic Neighbor Embedding).

In [ ]:
%%time
projector = TSNE(n_components=2, random_state=0)
train_2d = projector.fit_transform(X_train_real_scaled) 
plt.scatter(train_2d[y_train == 1, 0], train_2d[y_train == 1, 1], color='red', alpha = 0.5)
plt.scatter(train_2d[y_train == 0, 0], train_2d[y_train == 0, 1], color='blue', alpha = 0.1)
plt.show()

# Трансформация признаков.

Теперь рассмотрим способы преобразования признаков. Существует достаточно много различных способов трансформации признаков, которые позволяют при помощи линейных методов получать более сложные разделяющие поверхности. Самым базовым является полиномиальное преобразование признаков. Его идея заключается в том, что помимо самих признаков вы дополнительно включаете набор все полиномы степени $p$, которые можно из них построить. Для случая $p=2$ преобразование выглядит следующим образом:

$$ \phi(x_i) = [x_{i,1}^2, ..., x_{i,D}^2, x_{i,1}x_{i,2}, ..., x_{i,D-1}x_{i,D}, x_{i,1}, ..., x_{i,D}, 1] $$

Рассмотрим принцип работы данных признаков на данных, сэмплированных из гауссиан:

In [ ]:
np.random.seed(0)
# Сэмплируем данные из первой гауссианы
data_0 = np.random.multivariate_normal([0,0], [[0.5,0],[0,0.5]], size=40)
# И из второй
data_1 = np.random.multivariate_normal([0,1], [[0.5,0],[0,0.5]], size=40)
# На обучение берём 20 объектов из первого класса и 10 из второго
example_data_train = np.vstack([data_0[:20,:], data_1[:10,:]])
example_labels_train = np.concatenate([np.zeros((20)), np.ones((10))])
# На тест - 20 из первого и 30 из второго
example_data_test = np.vstack([data_0[20:,:], data_1[10:,:]])
example_labels_test = np.concatenate([np.zeros((20)), np.ones((30))])
# Задаём координатную сетку, на которой будем вычислять область классификации
xx, yy = np.meshgrid(np.arange(-3, 3, 0.02), np.arange(-3, 3, 0.02))

# Инициализируем класс, который выполняет преобразование
transform = PolynomialFeatures(2)
# Обучаем преобразование на обучающей выборке, применяем его к тестовой
example_data_train_poly = transform.fit_transform(example_data_train)
example_data_test_poly = transform.transform(example_data_test)
# Обращаем внимание на параметр fit_intercept=False
optimizer = GridSearchCV(LR(class_weight='balanced', fit_intercept=False), param_grid, cv=cv, n_jobs=-1)
optimizer.fit(example_data_train_poly, example_labels_train)
Z = optimizer.predict(transform.transform(np.c_[xx.ravel(), yy.ravel()])).reshape(xx.shape)
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)
plt.scatter(data_0[:,0], data_0[:,1], color='red')
plt.scatter(data_1[:,0], data_1[:,1], color='blue')
plt.title('Balanced classes')
plt.show()

Какие выводы можно сделать, глядя на получившийся график?

**Задание.** Выведите число признаков в новой модели.

In [ ]:
# место для вашего кода

Но при этом одновременно данный метод способствует более сильной способности модели к переобучению из-за быстрого роста числа признаком с увеличением степени $p$. Рассмотрим пример с $p=11$:

In [ ]:
transform = PolynomialFeatures(11)
example_data_train_poly = transform.fit_transform(example_data_train)
example_data_test_poly = transform.transform(example_data_test)
optimizer = GridSearchCV(LR(class_weight='balanced', fit_intercept=False), param_grid, cv=cv, n_jobs=-1)
optimizer.fit(example_data_train_poly, example_labels_train)
Z = optimizer.predict(transform.transform(np.c_[xx.ravel(), yy.ravel()])).reshape(xx.shape)
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Pastel2)
plt.scatter(data_0[:,0], data_0[:,1], color='red')
plt.scatter(data_1[:,0], data_1[:,1], color='blue')
plt.title('Corrected class weights')
plt.show()

**Задание**. Выведите количество признаков в данной модели.

In [ ]:
# место для вашего кода

Обратили ли вы внимание на то, что модель переобучилась? Посмотрите раздел "Понимание регуляризации" в следующей [статье](https://msdn.microsoft.com/ru-ru/magazine/dn904675.aspx).

**Задание**. Можете ли вы своими словами сформулировать что такое переобучение?

## Шаг 6. Трансформация вещественных признаков.

1. Реализуйте по аналогии с примером преобразование вещественных признаков модели при помощи полиномиальных признаков степени 2 и 3.
2. Постройте логистическую регрессию на новых данных, одновременно подобрав оптимальные гиперпараметры. Обращаем внимание, что в преобразованных признаках уже присутствует столбец, все значения которого равны 1, поэтому обучать дополнительно значение $b$ не нужно, его функцию выполняет один из весов $w$. В связи с этим во избежание линейной зависимости в датасете, в вызов класса логистической регрессии требуется передавать параметр fit_intercept=False. 
3. Получите AUC ROC на тесте и сравните данный результат с использованием обычных признаков.

In [ ]:
# место для вашего кода

In [ ]:
%%time
# место для вашего кода

In [ ]:
%%time
# место для вашего кода

**Задание**. Какое возведение в степень показывает себя лучше (1,2 или 3)?

## Регрессия Lasso.
К логистической регрессии также можно применить L1-регуляризацию (Lasso), вместо регуляризации L2, которая будет приводить к отбору признаков. Вам предлагается применить L1-регуляцию к исходным признакам и проинтерпретировать полученные результаты (применение отбора признаков к полиномиальным так же можно успешно применять, но в нём уже будет отсутствовать компонента интерпретации, т.к. смысловое значение оригинальных признаков известно, а полиномиальных - уже может быть достаточно нетривиально). Для вызова логистической регрессии с L1-регуляризацией достаточно передать параметр penalty='l1' в инициализацию класса.

## Шаг 7. Отбор признаков при помощи регрессии Lasso.

1. Обучите регрессию Lasso на стратифицированных отмасштабированных выборках, используя балансировку классов при помощи весов.
2. Получите ROC AUC регрессии, сравните его с предыдущими результатами.

In [ ]:
X_train = np.hstack([X_train_real_scaled, X_train_cat_oh])
X_test = np.hstack([X_test_real_scaled, X_test_cat_oh])
param_grid['penalty'] = ['l1']
model = analyze_cv(X_train, y_train, X_test, y_test, param_grid, cv)

**Задание**. Выведите количество вещественных признаков, которые имеют нулевые веса в итоговой модели. Выведите количество признаков порожденных от категориальных,  которые имеют нулевые веса в итоговой модели.

In [ ]:
# место для вашего кода

Уже сейчас видно, что наши признаки содержат много лишнего. Будем убирать малозначимые признаки с помощью преобразователя SelectFromModel. После этого сравним результаты обучения на исходном наборе и отфильтрованном.

In [ ]:
selector = SelectFromModel(model, prefit=True)
indices = selector.get_support()
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
param_grid['penalty'] = ['l2']  # тут нужно вернуть этот параметр в исходное состояние
model = analyze_cv(X_train, y_train, X_test, y_test, param_grid, cv)

**Задание**. Выведите сколько всего признаков осталось.

In [ ]:
# место для вашего кода

**Задание**. Найдите в документации sklearn информацию о том как отбираются признаки с помощью SelectFromModel. Какое значение параметра threshold мы использовали?

**Задание**.
1. Из множества вещественных признаков отфильтровать малозначимые.
2. Визуализировать это отфильтрованное множество вещественных признаков.
3. Построить вещественные признаки степени 2.
4. Обучить модель на совокупности признаков из п2 и отфильтрованных категориальных. Сравнить ее с предыдущими моделями.

In [ ]:
%%time
# место для вашего кода

In [ ]:
%%time
# место для вашего кода